# AutoPrognosis classification

Welcome! This tutorial will walk you through the steps of selecting a model for a classification task using AutoPrognosis.

### Setup

In [4]:
# stdlib
import json
import warnings

# third party
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

### Import ClassifierStudy

ClassifierStudy is the engine that learns an ensemble of pipelines and their hyperparameters automatically.

In [5]:
# autoprognosis absolute
from autoprognosis.studies.classifiers import ClassifierStudy
from autoprognosis.hooks import DefaultHooks, WandbHook

### Load the target dataset

AutoPrognosis expects pandas.DataFrames as input.

For this example, we will use the [Breast Cancer Wisconsin Dataset](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)).

In [6]:
# stdlib
from pathlib import Path

X, Y = load_breast_cancer(return_X_y=True, as_frame=True)

df = X.copy()
df["target"] = Y

### Create the classifier

While AutoPrognosis provides default plugins, it allows the user to customize the plugins for the pipelines.

You can see the supported plugins below:

In [7]:
# List the available plugins

# autoprognosis absolute
from autoprognosis.plugins import Plugins

print(json.dumps(Plugins().list_available(), indent=2))

{
  "imputer": {
    "default": [
      "EM",
      "gain",
      "hyperimpute",
      "ice",
      "mean",
      "median",
      "mice",
      "missforest",
      "most_frequent",
      "nop",
      "sinkhorn",
      "softimpute"
    ]
  },
  "prediction": {
    "classifier": [
      "adaboost",
      "bagging",
      "bernoulli_naive_bayes",
      "catboost",
      "decision_trees",
      "extra_tree_classifier",
      "gaussian_naive_bayes",
      "gaussian_process",
      "gradient_boosting",
      "hist_gradient_boosting",
      "knn",
      "lda",
      "lgbm",
      "linear_svm",
      "logistic_regression",
      "multinomial_naive_bayes",
      "neural_nets",
      "perceptron",
      "qda",
      "random_forest",
      "ridge_classifier",
      "tabnet",
      "xgboost"
    ],
    "regression": [
      "bayesian_ridge",
      "catboost_regressor",
      "kneighbors_regressor",
      "linear_regression",
      "mlp_regressor",
      "neural_nets_regression",
      "random_fore

We will set a few custom plugins for the pipelines and create the classifier study.

In [12]:
workspace = Path("workspace")
workspace.mkdir(parents=True, exist_ok=True)

study_name = "classification_example"

study = ClassifierStudy(
    study_name=study_name,
    dataset=df,  # pandas DataFrame
    target="target",  # the label column in the dataset
    num_iter=10,  # DELETE THIS LINE FOR BETTER RESULTS. how many trials to do for each candidate. Default: 50
    num_study_iter=1,  # DELETE THIS LINE FOR BETTER RESULTS. how many outer iterations to do. Default: 5
    classifiers=[
        "logistic_regression",
        "lda",
        "qda",
    ],  # DELETE THIS LINE FOR BETTER RESULTS.
    workspace=workspace,
    hooks=DefaultHooks().append(WandbHook()),
)

### Search for the optimal ensemble


In [13]:
study.run()

WeightedEnsemble(models=[<autoprognosis.plugins.pipeline.fast_ica_feature_normalizer_data_cleanup_lda object at 0x000002BE32366140>,
                         <autoprognosis.plugins.pipeline.fast_ica_feature_normalizer_data_cleanup_qda object at 0x000002BE32398250>,
                         <autoprognosis.plugins.pipeline.fast_ica_feature_normalizer_data_cleanup_logistic_regression object at 0x000002BE32367A90>],
                 weights=[0.2352941175086505, 0.470588235017301,
                          0.29411764688581316])

In [10]:
# stdlib
import pprint

# autoprognosis absolute
from autoprognosis.utils.serialization import load_model_from_file
from autoprognosis.utils.tester import evaluate_estimator

output = workspace / study_name / "model.p"

model = load_model_from_file(output)

metrics = evaluate_estimator(model, X, Y)

print(f"Model {model.name()} ")
print("Score: ")

pprint.pprint(metrics)

Model 0.1874999998828125 * (pca->maxabs_scaler->data_cleanup->lda) + 0.624999999609375 * (data_cleanup->logistic_regression) + 0.1874999998828125 * (pca->maxabs_scaler->data_cleanup->qda) 
Score: 
{'raw': {'accuracy': (0.9525666016894087, 0.00831882156526754),
         'aucprc': (0.9923907993530281, 0.004491214387734272),
         'aucroc': (0.9892418939406747, 0.0047820586713733714),
         'f1_score_macro': (0.9487085349863671, 0.008984798950139166),
         'f1_score_micro': (0.9525666016894087, 0.00831882156526754),
         'f1_score_weighted': (0.9522847627675723, 0.008405935546129645),
         'kappa': (0.8974702024409966, 0.017942781976667134),
         'kappa_quadratic': (0.8974702024409966, 0.017942781976667134),
         'mcc': (0.8984245080218889, 0.017605200083484915),
         'precision_macro': (0.954473104175413, 0.008611940788115212),
         'precision_micro': (0.9525666016894087, 0.00831882156526754),
         'precision_weighted': (0.9529458849052901, 0.0081736

## Serialization

In [11]:
# autoprognosis absolute
from autoprognosis.utils.serialization import load_from_file, save_to_file

out = workspace / "tmp.bkp"
# Fit the model
model.fit(X, Y)

# Save
save_to_file(out, model)

# Reload
loaded_model = load_from_file(out)

print(loaded_model.name())

assert loaded_model.name() == model.name()

out.unlink()

0.1874999998828125 * (pca->maxabs_scaler->data_cleanup->lda) + 0.624999999609375 * (data_cleanup->logistic_regression) + 0.1874999998828125 * (pca->maxabs_scaler->data_cleanup->qda)


## Congratulations!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement towards Machine learning and AI for medicine, you can do so in the following ways!

### Star AutoPrognosis on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the tools we're building.

- [Star AutoPrognosis](https://github.com/vanderschaarlab/autoprognosis)
- [Star HyperImpute](https://github.com/vanderschaarlab/hyperimpute)
